In [1]:
from textmining_utility.annotator.pipeline.pipeline_base import Pipeline
import pandas as pd

import pandas as pd
import numpy as np
import sys, os
import importlib
import textmining.cluster_analysis as cluster_analysis
import textmining.lexicons as lexicons
import textmining.machine_learning as machine_learning
import textmining.significance_testing as significance_testing
import iesta.loader as loader
import iesta.properties as prop  
import iesta.processor as proc  
import iesta.feature_extractor as fe

from collections import Counter

importlib.reload(cluster_analysis)
importlib.reload(lexicons)
importlib.reload(machine_learning)
importlib.reload(significance_testing)
importlib.reload(loader)
importlib.reload(prop)
importlib.reload(proc)
importlib.reload(fe)


<module 'iesta.feature_extractor' from 'c:\\users\\elba_ro\\documents\\projects\\conf22-style-transfer\\iesta\\feature_extractor.py'>

In [2]:
liberal_each_dataloader = IESTAData(ideology=properties.LIBERAL_IDEOLOGY, methodology=METHODOLOGY.EACH)

liberal_each_dataloader.load()



In [3]:
liberal_arguments_df.head()
#conservative_arguments_df

,id,debate_id,p_name,effects,effect_count,top_effect,effect,category,round,argument
0,.-.-.-Ha-YOURE-GOING-DOWN-BEEM0R/1/|Logical-Ma...,.-.-.-Ha-YOURE-GOING-DOWN-BEEM0R/1/,Logical-Master,ineffective,"b'{""ineffective"": 1}'",ineffective,ineffective,Miscellaneous,0,\n \r\nRESOLVED: It is impossible not to cons...
1,.-.-.-Ha-YOURE-GOING-DOWN-BEEM0R/1/|Logical-Ma...,.-.-.-Ha-YOURE-GOING-DOWN-BEEM0R/1/,Logical-Master,ineffective,"b'{""ineffective"": 1}'",ineffective,ineffective,Miscellaneous,1,"\n \r\nOkay, just now reading the round, it w..."
2,.-.-.-Ha-YOURE-GOING-DOWN-BEEM0R/1/|Logical-Ma...,.-.-.-Ha-YOURE-GOING-DOWN-BEEM0R/1/,Logical-Master,ineffective,"b'{""ineffective"": 1}'",ineffective,ineffective,Miscellaneous,2,\n \r\n--->RE:Since my opponent has left it u...
3,.-.-.-Ha-YOURE-GOING-DOWN-BEEM0R/1/|Logical-Ma...,.-.-.-Ha-YOURE-GOING-DOWN-BEEM0R/1/,Logical-Master,ineffective,"b'{""ineffective"": 1}'",ineffective,ineffective,Miscellaneous,3,\n \r\nCONSCIOUS SECTION: \n \r\nLadies and...
4,.-.-.-Ha-YOURE-GOING-DOWN-BEEM0R/1/|beem0r,.-.-.-Ha-YOURE-GOING-DOWN-BEEM0R/1/,beem0r,ineffective,"b'{""ineffective"": 1}'",ineffective,ineffective,Miscellaneous,0,\n \r\nI would eplain the vast differences be...


In [4]:
class SamplePipeline(Pipeline):

     def process_input(self) -> list:
        processed = []
        txt_df = self.input[["id", "argument"]].copy()
        txt_df =txt_df.rename(columns={"id": "input_id", "argument": "text"},)
        
        for idx, row in txt_df.iterrows():
            processed.append((row.text, {"input_id": row.input_id}))

        return processed

     def init_and_run(self):
        
        self.add_annotation_pipe(name = "senter",                  save_output= False,is_spacy=True, is_native=True)
        self.add_annotation_pipe(name = "EmotionPipeOrchestrator", save_output= True, is_spacy=True)
        self.add_annotation_pipe(name = "HedgePipeOrchestrator",   save_output= True, is_spacy=True)
        self.add_annotation_pipe(name = "ToxicityOrchestrator",    save_output= True, is_spacy=True)

        # annotate the input
        #self.set_spacy_language_model("en_core_web_md")
        self.annotate()
        # save annotations when "save_output" is set to True
        self.save()

In [ ]:
import pandas  as pd
test_df = pd.DataFrame([{"input_id": 1, "text": "This is a very hard time, I am devastated!"},
                         {"input_id": 2, "text": "You can not get your tiny brain to work on this so stupid!!"},
                         {"input_id": 3, "text": "The amonium Nitrate was sitting there for ages."},
                         {"input_id": 4, "text": "I love you and I love how you look"}
                       ])

In [ ]:
pipeline = SamplePipeline(save_output= True,
                 out_path = "../data/extracted_features/liberal_transformers_features.parquet")
pipeline.set_input(liberal_arguments_df)
#pipeline.set_input(test_df)
pipeline.init_and_run()

textmining-utility INFO     adding pipe with name EmotionPipeOrchestrator
textmining-utility DEBUG    loading textmining_utility.annotator.pipe.linguistic.emotion
textmining-utility INFO     orchestrator was initialized successfully
textmining-utility INFO     adding pipe with code emotion_hartmann_component
textmining-utility INFO     adding pipe with name HedgePipeOrchestrator
textmining-utility DEBUG    loading textmining_utility.annotator.pipe.linguistic.hedge
textmining-utility INFO     orchestrator was initialized successfully
textmining-utility INFO     adding pipe with code hedge_component
textmining-utility INFO     adding pipe with name ToxicityOrchestrator
textmining-utility DEBUG    loading textmining_utility.annotator.pipe.linguistic.toxicity
textmining-utility INFO     orchestrator was initialized successfully
textmining-utility INFO     adding pipe with code toxicity_component
textmining-utility INFO     Defining pipe default and spacy stacks
Some weights of the model ch

In [ ]:
pipeline.out_df.head()

In [ ]:
pipeline_cons = SamplePipeline(save_output= True,
                 out_path = "../data/extracted_features/conservative_transformers_features.parquet")
pipeline_cons.set_input(conservative_data)
pipeline_cons.init_and_run()

In [ ]:
pipeline_cons.processed_input#annotated_artifacts

In [ ]:
pipeline_cons.out_df

In [ ]:
lib_transformers_df = pd.read_parquet("../data/extracted_features/liberal_transformers_features.parquet")

lib_transformers_df.head()
lib_transformers_df['hedge_dominant'].value_counts().to_frame().plot(kind="bar")
lib_transformers_df['emotion_hartmann_label'].value_counts().to_frame().plot(kind="bar")
lib_transformers_df['toxicity_dominant'].value_counts().to_frame().plot(kind="bar")

lib_transformers_df[["emotion_hartmann_anger",
                     "emotion_hartmann_disgust",
                    "emotion_hartmann_fear", 
                    "emotion_hartmann_joy",
                    "emotion_hartmann_neutral",
                    "emotion_hartmann_surprise",
                    "emotion_hartmann_sadness"]].plot.hist(bins=12, alpha=0.5)